https://github.com/inhovation97/personal_project/blob/main/pytorch/pytorch_project_Image_preprocessing.ipynb

https://aigong.tistory.com/182

In [6]:
!pip install -U ray


     |████████████████████████████████| 57.6 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 173 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 299 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 5.9 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [7]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from glob import glob
import json
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn import metrics
from tqdm import tqdm_notebook as tqdm
import ray
%matplotlib inline

In [8]:
import torch
import torchvision
from torchvision import transforms # 이미지 데이터 transform
from torch.utils.data import DataLoader # 이미지 데이터 로더
import os
import PIL

In [9]:
device = 'cuda0' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(2022)
if device == 'cuda0':
    torch.cuda.manual_seed_all(2022)

In [10]:
''' 경로 저장 '''
img_paths = []
img_groups = []
coor_paths = []

sub_list = sorted([sub for sub in glob("../data/dataset/train/*/")])

for sub in sub_list:
    tmp_img_path = []
    tmp_json_path = []
    
    for img_path in glob(sub + '*.jpg'):
        tmp_img_path.append(img_path)
        img_groups.append(sub.split('/')[4])
    for coor_path in glob(sub + '*.json'):
        tmp_json_path.append(coor_path)
    img_paths += sorted(tmp_img_path)
    coor_paths += sorted(tmp_json_path)

In [11]:
len(img_paths)

11438

In [16]:
239 * 48

11472

In [14]:
int(len(img_paths) / 48) + 1

239

In [17]:
[1,2,3,4,5][2:4]

[3, 4]

In [18]:
''' 병렬처리 '''
ray.init()

@ray.remote
def get_data(path_list):
    img_list = []
    for path in path_list:
        with open(path, 'rb') as i:
            img = np.array( PIL.Image.open(i).getdata() )
            img_list.append(img)
    return img_list


# task 만들기

chunk_size = 48
chunk_count = int(len(img_paths) / chunk_size) + 1

task_list = []
for idx in range(chunk_size):
    img_paths_sel = img_paths[idx*chunk_size:(idx+1)*chunk_size]
    task_list.append(get_data.remote(img_paths_sel))

result_list = ray.get(task_list)

ValueError: The configured object store size (57.101312 GB) exceeds /dev/shm size (57.101312 GB). This will harm performance. Consider deleting files in /dev/shm or increasing its size with --shm-size in Docker. To ignore this warning, set RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE=1.

In [8]:
img_list = []
for path in tqdm(img_paths):
    with open(path, 'rb') as i:
        img = PIL.Image.open(i).getdata()
    img_list.append( np.array(img))

<ipython-input-8-b0200210c0cb>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for path in tqdm(img_paths):


  0%|          | 0/11438 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
img_numpy = np.stack(img_list)
np.save('img_numpy.npy',img_numpy)

In [ ]:
totensor = transforms.ToTensor()

img_t = totensor( temp )

In [33]:
np.array(img)

array([[79, 79, 79],
       [79, 79, 79],
       [79, 79, 79],
       ...,
       [97, 97, 97],
       [96, 96, 96],
       [96, 96, 96]])

In [22]:
temp.shape

(11438,)

In [23]:
img_list[0]

array(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7F073EE05C40>,
      dtype=object)

In [27]:
np.asarray(img.getdata())

ValueError: seek of closed file

In [28]:
img.getdata()

ValueError: seek of closed file